In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

df = pd.read_csv("input.csv", delimiter=",")
df = df.drop(columns=["name","description","temperament"]) # Retira-se nome, descrição e temperamento, pois são irrelevantes para o processamento

# Uma breve pesquisa sobre as raças mostra que estão na classe "Miscellaneous Class" estão contidos nos do grupo "Foundation Stock Service", porém 
# esperam reconhecimento de uma organização, então decidi agrupa-los em um grupo só
df["group"] = df["group"].replace(["Foundation Stock Service","Miscellaneous Class"],"Foundation and miscellaneous")

#### Ao ler os dados, é notavel que os valores para: grooming_frequency, shedding, energy_level, trainability e demeanor são equivalentes a sua categoria, logo, pode-se ignorar todas as colunas que terminem com "category" visto que são dados redundantes

In [ ]:
df = df.drop(columns=["grooming_frequency_category","shedding_category","energy_level_category","trainability_category","demeanor_category"])
df

#### Devido a quase 30% dos dados estarem sem o atributo "popularity" decidi por não incluir esse atributo

In [ ]:
df["popularity"] = pd.to_numeric(df["popularity"], errors="coerce")
median = df["popularity"].median(skipna=True)
df["popularity"] = df["popularity"].fillna(median)


In [ ]:
#le = preprocessing.LabelEncoder() #transforma atributo 'group' em número
#df["group"] = le.fit_transform(df["group"])
#df
        

#### Plotagem do gráfico de correlação

In [ ]:
corr= df.drop(columns="group").corr()
corr

#### Pelo gráfico de correlação, é notável que há uma correlação entre min e max height e weight, assim como min e max expectancy. Irei agrupa-los

In [ ]:
# Size define a junção das médias das alturas com a média dos pesos

df["size"] = (df["min_height"] + df["max_height"])/2 + (df["min_weight"] + df["max_weight"])/2
df["avg_expectancy"] = (df["min_expectancy"] + df["max_expectancy"])/2
df = df.drop(columns=["min_height","min_weight","min_expectancy","max_height","max_weight","max_expectancy"])
df

In [ ]:
corr = df.drop(columns="group").corr()
corr

#### Com os agrupamentos feitos, os atributos tem pouquissima correlação entre si.

In [ ]:

print('Valores faltantes:\n', df.isnull().sum())

In [ ]:
# No conjunto de dados, muitos dos dados que possuem atributos faltantes, possuem mais de um atributo faltante, e apesar de ter quase 90 atributos faltantes
# Apenas 41 Valores serão perdidos com o expurgo total dos dados incompletos
df["trainability_value"] = pd.to_numeric(df["trainability_value"], errors="coerce")
median = df["trainability_value"].median(skipna=True)
df["trainability_value"] = df["trainability_value"].fillna(median)

df["demeanor_value"] = pd.to_numeric(df["demeanor_value"], errors="coerce")
median = df["demeanor_value"].median(skipna=True)
df["demeanor_value"] = df["demeanor_value"].fillna(median)

df["shedding_value"] = pd.to_numeric(df["shedding_value"], errors="coerce")
median = df["shedding_value"].median(skipna=True)
df["shedding_value"] = df["shedding_value"].fillna(median)

#print('Valores faltantes:\n', df.isnull().sum())

df = df.dropna(how ='any') #elimina todas as linhas com dados ausentes

#### Decido plotar o gráfico de cada um para buscar valores que não fazem sentido, para garantir a qualidade dos dados

In [ ]:
for column in df.columns:
    plt.figure()
    plt.plot(df.index, df[column], marker='o', linestyle='None')
    plt.title(column)
    plt.xlabel('Índice')
    plt.ylabel(column)
    plt.show()

#### Para a coluna 'energy_value' há apenas um com o valor 0.2, porém aparentemente é um outlier, visto que tem uma categoria só para ele, chamada 'couch potato', então não irei o retirar. É perceptivel também, que para o 'grupo 3', há uma menor quantidade de dados, portanto, irei aplicar o boosting. É notavel também que o atributo "avg_expectancy" quase não tem variação, então irei retira-lo, pois não agrega em nada para o modelo

In [ ]:
df = df.drop(columns="avg_expectancy")
#df = df.drop(columns="size")
# Selecionar todos os dados do grupo 3
demeanor_02_data = df[df["demeanor_value"] == 0.2]
demeanor_1_data = df[df["demeanor_value"] == 1]
grooming_1_data = df[df["grooming_frequency_value"] == 1]
shedding_1_data = df[df["shedding_value"] == 1.0 ]
shedding_08_data = df[df["shedding_value"] == 0.8 ]
energy_04_data = df[df["energy_level_value"] == 0.4]
group_3_data = df[df['group'] == 3]
# Duplicar os dados do grupo 3
df = pd.concat([df, group_3_data], ignore_index=True)
df = pd.concat([df, grooming_1_data], ignore_index=True)
df = pd.concat([df, demeanor_02_data], ignore_index=True)
df = pd.concat([df, demeanor_1_data], ignore_index=True)
df = pd.concat([df, shedding_1_data], ignore_index=True)
df = pd.concat([df, shedding_08_data], ignore_index=True)
df = pd.concat([df, energy_04_data], ignore_index=True)


df["group"].value_counts()



In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Excluir a coluna 'group' antes de normalizar
df_without_group = df.drop('group', axis=1)

# Criar um objeto MinMaxScaler
min_max_scaler = MinMaxScaler()

# Normalizar os dados usando MinMaxScaler
df_normalized = min_max_scaler.fit_transform(df_without_group)

# Criar um novo DataFrame com os dados normalizados
df_normalized = pd.DataFrame(df_normalized, columns=df_without_group.columns)


In [ ]:
df_normalized.to_csv("Processed_data.csv",index=False)
df["group"].to_csv("grupos.csv",index=False)